This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [ ]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = 'XQiSYVmGqJnAYNXxALEF'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [47]:
# First, import the relevant modules
import requests
import json

In [48]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url= 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?api_key=XQiSYVmGqJnAYNXxALEF'
r = requests.get(url)
r.status_code

200

In [49]:
# This is for three days in case the first day was a Saturday with no trading
url= 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?start_date=2017-01-01&end_date=2017-01-03&api_key=XQiSYVmGqJnAYNXxALEF'
r = requests.get(url)
r.status_code

200

In [50]:
r.text # we have a dictionary for which the value for the first key is also a dictionary
# column names is a list, and data is a list of lists

'{"dataset":{"id":10095370,"dataset_code":"AFX_X","database_code":"FSE","name":"Carl Zeiss Meditec (AFX_X)","description":"Stock Prices for Carl Zeiss Meditec (AFX) from the Frankfurt Stock Exchange.\\u003cbr\\u003e\\u003cbr\\u003eTrading System: Xetra\\u003cbr\\u003e\\u003cbr\\u003eISIN: DE0005313704","refreshed_at":"2019-01-29T22:30:13.825Z","newest_available_date":"2019-01-29","oldest_available_date":"2000-06-07","column_names":["Date","Open","High","Low","Close","Change","Traded Volume","Turnover","Last Price of the Day","Daily Traded Units","Daily Turnover"],"frequency":"daily","type":"Time Series","premium":false,"limit":null,"transform":null,"column_index":null,"start_date":"2017-01-01","end_date":"2017-01-03","data":[["2017-01-03",35.9,35.93,35.34,35.48,null,70618.0,2515473.0,null,null,null],["2017-01-02",34.99,35.94,34.99,35.8,null,44700.0,1590561.0,null,null,null]],"collapse":null,"order":null,"database_id":6129}}'

# Convert the returned JSON object into a Python dictionary.

In [51]:
r.json()['dataset']['column_names'] 
# r.json(), where r is the "Response" from the Requests call, converts the string JSON file to a Python dictionary
# so 'Open' is Pythonic col 1 (need max and min): how to extract this as a Python list?
#'max change HERE is: 'High' minus 'Low' = (col 2 - col 3)
# closing price change vs prior closing price
# average trading volume for year: col 6

['Date',
 'Open',
 'High',
 'Low',
 'Close',
 'Change',
 'Traded Volume',
 'Turnover',
 'Last Price of the Day',
 'Daily Traded Units',
 'Daily Turnover']

In [52]:
r.json()['dataset']['data'] [0:3] #Hmm Notice that the last day is listed first !!!(so inverse chronological)

[['2017-01-03',
  35.9,
  35.93,
  35.34,
  35.48,
  None,
  70618.0,
  2515473.0,
  None,
  None,
  None],
 ['2017-01-02',
  34.99,
  35.94,
  34.99,
  35.8,
  None,
  44700.0,
  1590561.0,
  None,
  None,
  None]]

In [53]:
# So the Open price for the first and the second days is given by:
r.json()['dataset']['data'][0][1] 
r.json()['dataset']['data'][1][1]

34.99

In [54]:
opens= []  #replace with a list comprehension (as used below)
for i in range(2):
    opens.append(r.json()['dataset']['data'][i][1])
opens

[35.9, 34.99]

# Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 

In [55]:
# This API call is for the entire year of 2017 (255 trading days)
url= 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?start_date=2017-01-01&end_date=2017-12-31&api_key=XQiSYVmGqJnAYNXxALEF'
r = requests.get(url)
r.status_code

200

In [56]:
days= len(r.json()['dataset']['data'])
days

255

# Calculate what the highest and lowest opening prices were for the stock in this period.

In [57]:
opens_2017= []  # replace with a list comprehension as below (after making sure they yield identical results)
# the first two elements different from above because we are counting backwards as data listed in inverse chronological
for i in range(255):
    opens_2017.append(r.json()['dataset']['data'][i][1])
opens_2017[0:10] # Hmm ! first two days are different as inverse chronological


[51.76, 51.65, 51.45, 51.05, 51.16, 51.88, 52.73, 52.37, 52.7, 53.11]

In [90]:
opens_c= [r.json()['dataset']['data'][i][1] for i in range(255)]
# this repeats the output above, and generates the same output so is used hereafter.

opens_c[0:10]
opens_c[-10:]
opens_c= list(filter(None, opens_c)) # this code filters out missing values
# BTW the code ran before for the first 150 elements, so the missing element(s) musy have been after that
result= max(opens_c),min(opens_c)
result
# len(opens_c)  # 255
# max(opens_c) # typeError: '>' not supported between instances of 'NoneType' and 'float': one entry is type None 
# s= sorted(opens_c) # same error message as above: identical! (type None because Null/missing, rather than a float)
# s[0:5]
# opens_sorted= opens_c[0:150].sort()
# opens_sorted[0:5] #TypeError: 'NoneType' object is not subscriptable
# print(opens_sorted) # None !!
print('The highest and the lowest opening prices are respectively ' + str(result))

The highest and the lowest opening prices are respectively (53.11, 34.0)


In [59]:
# Highest Opening price: # these explorations are retained as a personal reminder of things I had tried first.
type(opens_2017)# indeed type is a list
type(opens_2017[0]) # indeed each individual element is a float rather than a string
max(opens_2017)

TypeError: '>' not supported between instances of 'NoneType' and 'float'

In [60]:
L1 = [3,7,36]
max(L1) # so max works on this list!
min(L1) # works also

3

In [61]:
# to find max daily High minus Low can we use Numpy?? (otherwise loop through both lists, as done below:)

# What was the largest change in any one day (based on High and Low price)?

In [62]:
highs= [r.json()['dataset']['data'][i][2] for i in range(255)]
highs[0:10]

[51.94, 51.82, 51.89, 51.5, 51.52, 52.04, 52.73, 52.75, 52.7, 53.54]

In [63]:
lows= [r.json()['dataset']['data'][i][3] for i in range(255)]
lows[0:10]

[51.45, 51.43, 50.76, 50.92, 50.9, 51.2, 51.07, 51.61, 51.64, 52.15]

In [64]:
changes= [highs[i]-lows[i]for i in range(255)]
changes[0:10]
max(changes)
print('The largest change in any one day (based on High and Low price) was: '+ str(max(changes)))

The largest change in any one day (based on High and Low price) was: 2.8100000000000023


# What was the largest change between any two days (based on Closing Price)?

In [65]:
closes= []  #replace with a list comprehension!  #column 4
for i in range(255):
    closes.append(r.json()['dataset']['data'][i][4])
closes[0:10]

[51.76, 51.6, 51.82, 51.32, 51.4, 51.27, 51.66, 52.62, 52.01, 52.67]

In [66]:
closes2= [r.json()['dataset']['data'][i][4] for i in range(255)]
closes2[0:10]
max(closes2) # this runs! (but why not opens?)

53.09

In [67]:
differences_c = [abs(closes2[i+1]- closes2[i]) for i in range(254)]
differences_c[0:5]
max(differences_c)

2.559999999999995

In [68]:
print('The largest absolute price change on a single day in 2017 was '+ str(max(differences_c)))

The largest absolute price change on a single day in 2017 was 2.559999999999995


# What was the average daily trading volume during this year?

In [69]:
volumes= []  #replace with a list comprehension! volume is the 6th Column
for i in range(255):
    volumes.append(r.json()['dataset']['data'][i][6])
volumes[0:10]

[34640.0,
 40660.0,
 57452.0,
 71165.0,
 120649.0,
 50587.0,
 137313.0,
 129733.0,
 204080.0,
 132981.0]

In [70]:
volumes_c= [r.json()['dataset']['data'][i][6] for i in range(255)]
volumes_c[0:10]
avg_2017= sum(volumes_c)/len(volumes_c)
print('The average daily trading volume in 2017 was '+ str(avg_2017))

The average daily trading volume in 2017 was 89124.33725490196


In [71]:
sum(volumes)
len(volumes)
print('Average Daily Trading volume is '+ str(sum(volumes)/len(volumes)))

Average Daily Trading volume is 89124.33725490196


KeyError: 'data'

In [19]:
f= r.json()
type(f)

dict

In [ ]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

Here is an API call for FB stock data in CSV format:

https://www.quandl.com/api/v3/datasets/WIKI/FB/data.csv
Here is the same call, with some additional parameters appended:

https://www.quandl.com/api/v3/datasets/WIKI/FB/data.csv?column_index=4&exclude_column_names=true&rows=3&start_date=2012-11-01&end_date=2013-11-30&order=asc&collapse=quarterly&transform=rdiff
The second call gets FB stock data, but only column 4 (closing prices); it skips column names, truncates the data at 3 rows, selects only data between 2012-11-01 and 2013-11-30, arranges the dates in ascending order, down-samples daily data to quarterly, and computes percentage changes.

just append your API key to your call like this:

https://www.quandl.com/api/v3/datasets/WIKI/FB/data.csv?api_key=YOURAPIKEYHERE

You can customize the data you receive by passing optional parameters with your API call.  Here are some of the available options: Date Range	"start_date=yyyy-mm-dd"  and/or "end_date=yyyy-mm-dd"
